In [ ]:
import xarray as xr
from filmscope.util import convert_to_array_image_numbers
import numpy as np
from matplotlib import pyplot as plt
from filmscope.util import get_preview_image

In [10]:
path_to_data = "/media/Friday/Temporary/Clare/20250116_macaque_brain"
fluoro_filename = path_to_data + "/brain_20250116_162444_911.nc" #'/brain_20250116_162420_711.nc'
bright_filename = path_to_data + '/brain_20250116_162420_711.nc'
bright_filename = path_to_data + '/brain_20250116_162503_208.nc'


fluoro_cameras = [13, 15, 20, 22, 25, 27, 32, 34]
bright_cameras = [14, 16, 19, 21, 26, 28, 31, 33]

bright_cameras = convert_to_array_image_numbers(bright_cameras)
fluoro_cameras = convert_to_array_image_numbers(fluoro_cameras)

In [11]:
fluoro_dataset = xr.open_dataset(fluoro_filename)
bright_dataset = xr.open_dataset(bright_filename)



In [12]:
# this could definietly be better, 
# but start by taking only the necessary cameras from one of the dataset 
x_cams = np.unique(np.concatenate((fluoro_cameras[:, 0], bright_cameras[:, 0])))
y_cams = np.unique(np.concatenate((bright_cameras[:, 1], fluoro_cameras[:, 1])))
new_dataset = fluoro_dataset.loc[{"image_x": x_cams, "image_y": y_cams}]

In [ ]:
fluoro = new_dataset.loc[{"image_x": 3, "image_y": 2}].images.data
plt.figure()
plt.imshow(fluoro)

In [14]:
for var in new_dataset.data_vars:
    coords = new_dataset[var].coords 
    if not ("image_y" in coords and "image_x" in coords):
        continue
    for cam_pair in bright_cameras:
        bright_d = bright_dataset[var].loc[{"image_x": cam_pair[0], "image_y": cam_pair[1]}]
        new_dataset[var].loc[{"image_x": cam_pair[0], "image_y": cam_pair[1]}] = bright_d

In [ ]:
bright = new_dataset.loc[{"image_x": 2, "image_y": 2}].images.data
plt.imshow(bright)

In [16]:
# then we can save! 
save_filename = path_to_data + '/combo_fluoro_bright_dataset_0.nc'
new_dataset.to_netcdf(save_filename)

In [ ]:
# and code to display
image_filename = save_filename
image_numbers = [13, 14, 15, 16, 
                 19, 20, 21, 22,
                 25, 26, 27, 28,
                 31, 32, 33, 34]
downsample = 3
img = get_preview_image(image_filename, ensure_grayscale=False, image_numbers=image_numbers, downsample=downsample)

# this is not effective, we should change this...
xv = int(4096 / downsample) 
yv = int(2 * 3072 / downsample)
img = img[xv:-xv, yv:-yv]

plt.imshow(img) 
ax = plt.gca()
ax.set_xticks([])
ax.set_yticks([])